# Problema

Uma consideração importante no estudo de transferência de calor é a de determinar a distribuição de
temperatura assintótica de uma placa fina quando a temperatura em seu bordo é conhecida. Suponha que a placa na
Figura 2 represente uma seção transversal de uma barra de metal, com fluxo de calor desprezível na direção
perpendicular à placa. Sejam $T_1, T_2, \dots, T_6$ as temperaturas em seis vértices interiores do reticulado da Figura 1. A temperatura num vértice é aproximadamente igual à média dos quatro vértices vizinhos mais próximos - à esquerda, acima, à direita e abaixo. Por exemplo,

$$ T_1 = \frac{(10+20+T_2+T_4)}{4} \hspace{0.5cm} \text{ou} \hspace{0.5cm} 4T_1-T_2-T_4=30  $$

| <img src="figura2.png" width="300px"></img> |
|:--:|
|*Figura 1. Temperatura em seis vértices interiores do reticulado*|

**a)** Escreva um sistema de seis equações cuja solução forneça estimativas para as temperaturas $T_1, T_2, \dots, T_6$

**b)** Resolva o sistema linear obtido em **a)** utilizando o método de fatoração LU, sem e com pivotamento.

# Solução

## a) Equações

$$
T_1 = \frac{1}{4}(10 + 20 + T_2 + T_4) \\
T_2 = \frac{1}{4}(T_1 + 20 + T_3 + T_5) \\
T_3 = \frac{1}{4}(T_2 + 20 + 40 + T_6) \\
T_4 = \frac{1}{4}(10 + T_1 + T_5 + 20) \\
T_5 = \frac{1}{4}(T_4 + T_2 + T_6 + 20) \\
T_6 = \frac{1}{4}(T_5 + T_3 + 40 + 20)
$$

Logo,

$$ \begin{cases}
    4T_1 &-&  T_2 &+& 0T_3 &-&  T_4 &+& 0T_5 &+& 0T_6 &=& 30 \\
    -T_1 &+& 4T_2 &-& T_3  &+& 0T_4 &-& T_5  &+& 0T_6 &=& 20 \\
    0T_1 &-&  T_2 &+& 4T_3 &+& 0T_4 &+& 0T_5 &-&  T_6 &=& 60  \\
    -T_1 &+& 0T_2 &+& 0T_3 &+& 4T_4 &-& T_5  &+& 0T_6 &=& 30 \\
    0T_1 &-&  T_2 &+& 0T_3 &-&  T_4 &+& 4T_5 &-&  T_6 &=& 20 \\
    0T_1 &+& 0T_2 &-& T_3  &+& 0T_4 &-& T_5  &+& 4T_6 &=& 60
\end{cases} $$

Que resulta na matriz de coeficientes:

$$
\begin{bmatrix}
    4 & -1 & 0 & -1 & 0 & 0 \\
    -1 & 4 & -1 & 0 & -1 & 0 \\
    0 & -1 & 4 & 0 & 0 & -1 \\
    -1 & 0 & 0 & 4 & -1 & 0 \\
    0 & -1 & 0 & -1 & 4 & -1 \\
    0 & 0 & -1 & 0 & -1 & 4
\end{bmatrix}
$$


## b) [Fatoração LU](https://pt.wikipedia.org/wiki/Decomposi%C3%A7%C3%A3o_LU)

A fatoração de uma matriz $A$ é uma equação que expressa $A$ como o produto de duas ou mais matrizes. 

Na linguagem da ciência da computação, a expressão que representa $A$ na forma de um produto pode ser entendida como um processamento de dados, pois os dados são organizados em duas ou mais partes cujas estruturas, de alguma forma, são mais fáceis de lidar computacionalmente.

A fatoração LU é amplamente aplicada quando quer-se resolver uma sequência de equações, todas com a mesma matriz de coeficientes, por exemplo:

$$ Ax=b_1, \quad Ax=b_2, \quad \cdots, Ax=b_n. $$

Seu resultado é a descrição de $A$ como o produto de duas matrizes, $L$ e $U$, matrizes triangulares inferior e superior, respectivamente.

$$ A = LU $$

$$ 
\begin{bmatrix}
a_{11} & a_{12} & a_{13} & \dots & a_{1n} \\
a_{21} & a_{22} & a_{23} & \dots & a_{2n} \\
a_{21} & a_{32} & a_{33} & \dots & a_{3n} \\
\dots \\
a_{m1} & a_{m2} & a_{m3} & \dots & a_{mn} \\
\end{bmatrix}

= 

\begin{bmatrix}
l_{11} & 0 & 0 & \dots & 0 \\
l_{21} & l_{22} & 0 & \dots & 0  \\
l_{31} & l_{32} & l_{33} & \dots & 0  \\
\dots \\
l_{m1} & l_{m2} & l_{m3} & \dots & l_{mn} \\
\end{bmatrix}

\begin{bmatrix}
u_{11} & u_{12} & u_{13} & \dots & u_{1n} \\
0 & u_{22} & u_{23} & \dots & u_{2n} \\
0 & 0 & u_{33} & \dots & u_{3n} \\
\dots \\
0 & 0 & 0 & \dots & u_{mn} \\
\end{bmatrix}

$$

O algoritmo de decomposição LU é essencialmente uma forma modificada do algoritmo da eliminação de Gauss.
A maior diferença está no fato de que, se armazenarmos numa segunda matriz identidade $L$ os coeficientes aos quais as linhas da matriz original estão sendo multiplicadas, obtemos uma matriz triangular inferior.

$$
L = 
\begin{bmatrix}
1 & 0 & 0 & \dots & 0 \\
\frac{a_{21}}{a_{11}} & 1 & 0 & \dots & 0  \\
\frac{a_{31}}{a_{11}} & \frac{a_{32}}{a_{22}} & 1 & \dots & 0  \\
\dots \\
\frac{a_{m1}}{a_{11}} & \frac{a_{m2}}{a_{22}} & \frac{a_{m3}}{a_{33}} & \dots & 1\\
\end{bmatrix}
$$

Quando essa matriz $L$ é multiplicada pela matriz triangular superior resultante da eliminação de Gauss $U$, obtemos $A$.

Se para cada coluna da eliminação gaussiana definirmos a matriz $L_i$, também identidade, apenas com os coeficientes da coluna atual,
Também, podemos descrever cada passo da eliminação gaussiana $A_i$ como
$$ A_i = L_iA_{i-1} $$

Por consequência temos
$$ A_i = L_iA_{i-1} = L_iL_{i-1}A_{i-2} = L_iL_{i-1}L_{i-2}A_{i-3} = L_iL_{i-1}L_{i-2}\dots L_1A_0 $$

Isolando $A_0$, ou $A$,
$$
\begin{align*} 
A_i &= L_i A_{i-1} \\
L_i^{-1}A_i &= A_{i-1}  \\
L_i^{-1}A_i &= L_{i-1}A_{i-2} \\
L_{i-1}L_iA_i &= A_{i-2} \\
\dots \\
L_1^{-1}L_2^{-1} \dots L_i^{-1}A_i &= A_0 \\
(L_1L_2 \dots L_i)^{-1}A_i &= A_0 = A
\end{align*}
$$


In [4]:
import numpy as np
from IPython.display import display, Markdown, Latex
from furg_imef_verificador_respostas import Verificador

def verificar_resposta(x):
    verificador = Verificador()
    verificador.verificar_resposta(x)


In [89]:
A = np.array([
    [ 4, -1,  0, -1,  0,  0],
    [-1,  4, -1,  0, -1,  0],
    [ 0, -1,  4,  0,  0, -1],
    [-1,  0,  0,  4, -1,  0],
    [ 0, -1,  0, -1,  4, -1],
    [ 0,  0, -1,  0, -1,  4]
]).astype(np.float32)

b = np.array([30, 20, 60, 30, 20, 60])

### Sem pivotamento

In [90]:
# Criamos inicialmente um array de matrizes, com o item 0 sendo a matriz de coeficientes.
As = [A.copy()]

# Fazemos o mesmo com o item 0 sendo uma matriz identidade nas mesmas dimensões que A.
Ls = [np.eye(A.shape[0])]

# Para cada linha j de A
for j in range(As[0].shape[0]):
    # Selecionamos a matriz na posição j do nosso array de matrizes A.
    A_i = As[j]

    # Selecionamos também o valor equivalente à j na diagonal principal.
    a_nn = A_i[j,j]
    
    # Criamos uma cópia da matriz que estamos trabalhando ( A_i ) 
    A_j = A_i.copy()

    # E criamos uma nova matriz identidade. I_j
    L_n = np.eye(A_i.shape[0])

    # Para cada coluna à direita do valor da linha j, na matriz A_i
    for i in range(j+1, A_i.shape[1]):
        # Selecionamos o valor em i, j
        a_in = A_j[i,j]

        # Calculamos o coeficiente que reduz a linha à 0.
        l_in = -a_in/a_nn

        # Atualizamos o valor da matriz identidade atual para o coeficiente.
        L_n[i, j] = l_in
        
    # Multiplicamos a matriz de coeficientes pela matriz com os coeficientes calculados,
    # reduzindo os valores abaixo do pivô a 0.
    A_n = L_n@A_j
    
    # Adicionamos ao array de matrizes a matriz atual.
    As.append(A_n)

    # Fazemos o mesmo com a matriz de coeficientes.
    Ls.append(L_n)

L = np.eye(A.shape[0])
for L_i in Ls:
    L = L@L_i
L = np.linalg.inv(L)
U = As[-1]

display((L@U).round(0))
A

    

array([[ 4., -1.,  0., -1.,  0.,  0.],
       [-1.,  4., -1.,  0., -1.,  0.],
       [ 0., -1.,  4., -0.,  0., -1.],
       [-1.,  0., -0.,  4., -1.,  0.],
       [ 1., -1., -0., -1.,  4., -1.],
       [-0.,  1., -1.,  0., -1.,  4.]])

array([[ 4., -1.,  0., -1.,  0.,  0.],
       [-1.,  4., -1.,  0., -1.,  0.],
       [ 0., -1.,  4.,  0.,  0., -1.],
       [-1.,  0.,  0.,  4., -1.,  0.],
       [ 0., -1.,  0., -1.,  4., -1.],
       [ 0.,  0., -1.,  0., -1.,  4.]], dtype=float32)

## Versão reduzida do algoritmo

Como dito antes, a matriz $L$ pode ser encontrada armazenando os coeficientes do processo de eliminação numa matriz identidade. O algoritmo poderia então ser implementado como:

In [92]:
def lu_decomp(M):
    M = M.copy()
    L = np.eye(M.shape[0])
    for i in range(M.shape[0]):
        aii = M[i, i]
        for j in range(i+1, M.shape[0]):
            aij = M[i, j]
            coef = aij / aii
            M[j,:] = M[j,:]-M[i,:]*coef
            L[j, i] = coef

    return M, L

M = A
U, L = lu_decomp(M)

display(L.round(1), U.round(1))
display((L@U).round(2))

array([[ 1. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [-0.2,  1. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. , -0.3,  1. ,  0. ,  0. ,  0. ],
       [-0.2, -0.1, -0. ,  1. ,  0. ,  0. ],
       [ 0. , -0.3, -0.1, -0.3,  1. ,  0. ],
       [ 0. ,  0. , -0.3, -0. , -0.3,  1. ]])

array([[ 4. , -1. ,  0. , -1. ,  0. ,  0. ],
       [ 0. ,  3.8, -1. , -0.2, -1. ,  0. ],
       [ 0. ,  0. ,  3.7, -0.1, -0.3, -1. ],
       [ 0. ,  0. ,  0. ,  3.7, -1.1, -0. ],
       [ 0. ,  0. ,  0. ,  0. ,  3.4, -1.1],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  3.4]], dtype=float32)

array([[ 4., -1.,  0., -1.,  0.,  0.],
       [-1.,  4., -1.,  0., -1.,  0.],
       [ 0., -1.,  4.,  0.,  0., -1.],
       [-1., -0.,  0.,  4., -1.,  0.],
       [ 0., -1.,  0., -1.,  4., -1.],
       [ 0.,  0., -1., -0., -1.,  4.]])

### Com pivotamento

# Decomposição QR

In [18]:
import math 

def proj(u, v):
    return np.dot(u, v) / np.dot(u, u) * u

def QR_decomp(A):
    u = []
    for i in range(A.shape[1]):
        u.append(
            A[:,i] - np.sum([ proj(u[j], A[:, i]) for j in range(i) ], axis=0 )
        )

    Q = np.array([u[i]/math.sqrt(np.dot(u[i],u[i])) for i in range(len(u))]).T

    R = []
    for i in range(A.shape[0]):
        R.append(
            [ np.dot(Q[:,i], A[:,j])*(1-max(0, min(i-j,1))) for j in range(A.shape[1]) ]
        )
    R = np.array(R)

    return Q, R

# Resolve um sistema de equações cuja matriz de coeficientes é 
# do tipo triangular superiora.
def resolver_U(U, b):
    x = np.zeros(U.shape[1])
    for i in reversed(range(U.shape[0])):
        x[i]= ( b[i] - np.sum([U[i,j]*x[j] for j in range(i+1, U.shape[0])]) ) / U[i,i]
    return x


Q, R = QR_decomp(A)
display(R.round(2), (Q.T@b).round(2))
x = resolver_U(R, Q.T@b)

verificador.verificar_resposta(x)


array([[ 4.24, -1.89,  0.24, -1.89,  0.47,  0.  ],
       [-0.  ,  3.93, -1.92, -0.4 , -1.81,  0.51],
       [-0.  , -0.  ,  3.77, -0.08, -0.42, -1.86],
       [ 0.  , -0.  ,  0.  ,  3.78, -2.08,  0.28],
       [ 0.  ,  0.  ,  0.  , -0.  ,  3.32, -2.2 ],
       [ 0.  , -0.  ,  0.  , -0.  ,  0.  ,  3.06]])

array([16.5 ,  0.28, 41.5 , 27.71, 11.42, 83.11])

Resposta correta!


True